This code aggregates the product, brand and category customer behavior profiles for each month created from the notebook: Customer_Behavior_Profile_Month.ipynb for the customers that purchased (during the 7 months of data) which were identified on the notebook:Customer_Behavior_Profile_Analysis.ipynb 

In [1]:
# Loading basic needed libraries
import pandas as pd
import numpy as np
import gc
from functools import reduce
import datetime as dt
from datetime import date

# Loading libraries for S3 bucket connection
import boto3
import io
from io import StringIO,BytesIO, TextIOWrapper
import gzip

client = boto3.client('s3') 
resource = boto3.resource('s3') 

#### Creating full category affinity profile

In [2]:
# Reading customers who purchased
main_custs = pd.read_csv('s3://myaws-capstone-bucket/data/customers_of_focus.csv')
main_custs.nunique()

user_id                     1817173
total_view                     2424
total_cart_add                  484
total_purchases                 366
total_sessions                  306
total_spent                  387698
min_spent                     39053
max_spent                     57825
cust_retailer_age               213
days_since_last_activity        213
first_view_age                  214
days_since_last_view            214
first_cart_age                  214
days_since_last_cart            214
first_purchase_age              211
days_since_last_purchase        211
dtype: int64

In [3]:
# Reading customer overall profiles from S3
df_oct = pd.read_csv('s3://myaws-capstone-bucket/data/2019-Oct/customer_category_profile.csv')
df_nov = pd.read_csv('s3://myaws-capstone-bucket/data/2019-Nov/customer_category_profile.csv')
df_dec = pd.read_csv('s3://myaws-capstone-bucket/data/2019-Dec/customer_category_profile.csv')
df_jan = pd.read_csv('s3://myaws-capstone-bucket/data/2020-Jan/customer_category_profile.csv')
df_feb = pd.read_csv('s3://myaws-capstone-bucket/data/2020-Feb/customer_category_profile.csv')
df_mar = pd.read_csv('s3://myaws-capstone-bucket/data/2020-Mar/customer_category_profile.csv')
df_apr = pd.read_csv('s3://myaws-capstone-bucket/data/2020-Apr/customer_category_profile.csv')

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (13,14) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (13,14,15,16) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
# Concatanating dfs together to create one full customer profile for the 7 months
data_frames = [df_oct, df_nov, df_dec, df_jan, df_feb, df_mar, df_apr]
cust_profile = pd.concat(data_frames)
cust_profile.nunique()

user_id                  13699114
category                      973
view_count                   1409
cart_count                    293
purchase_count                223
session_count                 541
spent                      327552
min_category_spent          61661
max_category_spent          63376
median_category_spent      160131
avg_category_spent         383428
min_activity_date             213
max_activity_date             213
min_view_date                 213
max_view_date                 213
min_cart_date                 213
max_cart_date                 213
min_purchase_date             211
max_purchase_date             211
month                           7
dtype: int64

In [5]:
# Merging with customers who have made purchases as these will be our focus customers for the project
cust_profile = pd.merge(cust_profile, main_custs[['user_id']], on=["user_id"], how='inner')
cust_profile.nunique()

user_id                  1817173
category                     973
view_count                   975
cart_count                   281
purchase_count               223
session_count                255
spent                     327552
min_category_spent         61661
max_category_spent         63376
median_category_spent     160131
avg_category_spent        383428
min_activity_date            213
max_activity_date            213
min_view_date                213
max_view_date                213
min_cart_date                213
max_cart_date                213
min_purchase_date            211
max_purchase_date            211
month                          7
dtype: int64

In [6]:
# Rank column based on month customer shopped
cust_profile.loc[(cust_profile['month'] == '2019-Oct'),'month_rank'] = 7
cust_profile.loc[(cust_profile['month'] == '2019-Nov'),'month_rank'] = 6
cust_profile.loc[(cust_profile['month'] == '2019-Dec'),'month_rank'] = 5
cust_profile.loc[(cust_profile['month'] == '2020-Jan'),'month_rank'] = 4
cust_profile.loc[(cust_profile['month'] == '2020-Feb'),'month_rank'] = 3
cust_profile.loc[(cust_profile['month'] == '2020-Mar'),'month_rank'] = 2
cust_profile.loc[(cust_profile['month'] == '2020-Apr'),'month_rank'] = 1

In [7]:
cust_profile.nunique()

user_id                  1817173
category                     973
view_count                   975
cart_count                   281
purchase_count               223
session_count                255
spent                     327552
min_category_spent         61661
max_category_spent         63376
median_category_spent     160131
avg_category_spent        383428
min_activity_date            213
max_activity_date            213
min_view_date                213
max_view_date                213
min_cart_date                213
max_cart_date                213
min_purchase_date            211
max_purchase_date            211
month                          7
month_rank                     7
dtype: int64

In [8]:
# Saving results with month data added
cust_profile.to_csv('s3://myaws-capstone-bucket/data/category_profile_full_month_agg.csv',index=False)

In [9]:
# Transforming date columns to datetime type
cust_profile["max_activity_date"] = cust_profile["max_activity_date"].astype('datetime64[ns]')
cust_profile["min_activity_date"] = cust_profile["min_activity_date"].astype('datetime64[ns]')

cust_profile["min_view_date"] = cust_profile["min_view_date"].astype('datetime64[ns]')
cust_profile["max_view_date"] = cust_profile["max_view_date"].astype('datetime64[ns]')

cust_profile["min_cart_date"] = cust_profile["min_cart_date"].astype('datetime64[ns]')
cust_profile["max_cart_date"] = cust_profile["max_cart_date"].astype('datetime64[ns]')

cust_profile["max_purchase_date"] = cust_profile["max_purchase_date"].astype('datetime64[ns]')
cust_profile["min_purchase_date"] = cust_profile["min_purchase_date"].astype('datetime64[ns]')

In [10]:
# Grabbing max dates based on month rank
max_activity_dates = cust_profile.sort_values('max_activity_date',ascending = False).groupby('user_id').head(1)
max_activity_dates = max_activity_dates[['user_id','category','max_activity_date']]

max_view_dates = cust_profile.sort_values('max_view_date',ascending = False).groupby('user_id').head(1)
max_view_dates = max_view_dates[['user_id','category','max_view_date']]

max_cart_dates = cust_profile.sort_values('max_cart_date',ascending = False).groupby('user_id').head(1)
max_cart_dates = max_cart_dates[['user_id','category','max_cart_date']]

max_purchase_dates = cust_profile.sort_values('max_purchase_date',ascending = False).groupby('user_id').head(1)
max_purchase_dates = max_purchase_dates[['user_id','category','max_purchase_date']]

# merging_max_dates
data_frames = [max_activity_dates,max_view_dates, max_cart_dates, max_purchase_dates]
max_dates = reduce(lambda  left,right: pd.merge(left,right,on=['user_id','category'],how='outer'), data_frames)

In [11]:
max_dates.nunique()

user_id              1817173
category                 944
max_activity_date        213
max_view_date            213
max_cart_date            213
max_purchase_date        211
dtype: int64

In [12]:
# Grabbing min dates based on month rank
min_activity_dates = cust_profile.sort_values('min_activity_date',ascending = True).groupby('user_id').head(1)
min_activity_dates = min_activity_dates[['user_id','category','min_activity_date']]

min_view_dates = cust_profile.sort_values('min_view_date',ascending = True).groupby('user_id').head(1)
min_view_dates = min_view_dates[['user_id','category','min_view_date']]

min_cart_dates = cust_profile.sort_values('min_cart_date',ascending = True).groupby('user_id').head(1)
min_cart_dates = min_cart_dates[['user_id','category','min_cart_date']]

min_purchase_dates = cust_profile.sort_values('min_purchase_date',ascending = True).groupby('user_id').head(1)
min_purchase_dates = min_purchase_dates[['user_id','category','min_purchase_date']]

# merging_min_dates
data_frames = [min_activity_dates,min_view_dates, min_cart_dates, min_purchase_dates]
min_dates = reduce(lambda  left,right: pd.merge(left,right,on=['user_id','category'],how='outer'), data_frames)

In [13]:
min_dates.nunique()

user_id              1817173
category                 947
min_activity_date        213
min_view_date            213
min_cart_date            213
min_purchase_date        211
dtype: int64

In [14]:
# Obtaining max possible date from data 
column = df_apr["max_activity_date"].astype('datetime64[ns]')
max_date = column.max() 
# Transforming date columns to datetime format
max_dates[['max_activity_date','max_view_date','max_cart_date', 'max_purchase_date']] = max_dates[['max_activity_date','max_view_date','max_cart_date', 'max_purchase_date']].astype('datetime64[ns]')
min_dates[['min_activity_date','min_view_date','min_cart_date', 'min_purchase_date']] = min_dates[['min_activity_date','min_view_date','min_cart_date', 'min_purchase_date']].astype('datetime64[ns]')

customer_dates = pd.merge(max_dates, min_dates, on=["user_id","category"], how='outer')

In [15]:
customer_dates.nunique()

user_id              1817173
category                 959
max_activity_date        213
max_view_date            213
max_cart_date            213
max_purchase_date        211
min_activity_date        213
min_view_date            213
min_cart_date            213
min_purchase_date        211
dtype: int64

In [16]:
# Calculating days since each event and customer activity
customer_dates['cust_retailer_age'] = (max_date - customer_dates['min_activity_date']).dt.days
customer_dates['days_since_last_activity'] = (max_date - customer_dates['max_activity_date']).dt.days

customer_dates['first_view_age'] = (max_date - customer_dates['min_view_date']).dt.days
customer_dates['days_since_last_view'] = (max_date - customer_dates['max_view_date']).dt.days

customer_dates['first_cart_age'] = (max_date - customer_dates['min_cart_date']).dt.days
customer_dates['days_since_last_cart'] = (max_date - customer_dates['max_cart_date']).dt.days


customer_dates['first_purchase_age'] = (max_date - customer_dates['min_purchase_date']).dt.days
customer_dates['days_since_last_purchase'] = (max_date - customer_dates['max_purchase_date']).dt.days

customer_dates[['cust_retailer_age',
       'days_since_last_activity', 'first_view_age', 'days_since_last_view',
       'first_cart_age', 'days_since_last_cart', 'first_purchase_age',
       'days_since_last_purchase']] = customer_dates[['cust_retailer_age',
       'days_since_last_activity', 'first_view_age', 'days_since_last_view',
       'first_cart_age', 'days_since_last_cart', 'first_purchase_age',
       'days_since_last_purchase']].fillna(value=9999)# Replacing nan with 9999 to symbolize they have not done said action

customer_dates.head()

,user_id,category,max_activity_date,max_view_date,max_cart_date,max_purchase_date,min_activity_date,min_view_date,min_cart_date,min_purchase_date,cust_retailer_age,days_since_last_activity,first_view_age,days_since_last_view,first_cart_age,days_since_last_cart,first_purchase_age,days_since_last_purchase
0,649742534,2232732093077520756_construction.tools.light,2020-04-30,NaT,NaT,2020-04-30,2020-04-30,NaT,NaT,2020-04-30,0.0,0.0,9999.0,9999.0,9999.0,9999.0,0.0,0.0
1,549852898,2053013558920217191_computers.notebook,2020-04-30,NaT,NaT,NaT,NaT,NaT,NaT,NaT,9999.0,0.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0
2,549853909,2053013553325015316_appliances.kitchen.toster,2020-04-30,2020-04-30,NaT,NaT,NaT,NaT,NaT,NaT,9999.0,0.0,9999.0,0.0,9999.0,9999.0,9999.0,9999.0
3,513434454,2232732091718566220_appliances.kitchen.refrige...,2020-04-30,2020-04-30,NaT,NaT,NaT,NaT,NaT,NaT,9999.0,0.0,9999.0,0.0,9999.0,9999.0,9999.0,9999.0
4,618014976,2232732101063475749_appliances.environment.vacuum,2020-04-30,2020-04-30,2020-04-26,2020-04-26,NaT,NaT,2020-04-26,2020-04-26,9999.0,0.0,9999.0,0.0,4.0,4.0,4.0,4.0


In [17]:
customer_dates.nunique()

user_id                     1817173
category                        959
max_activity_date               213
max_view_date                   213
max_cart_date                   213
max_purchase_date               211
min_activity_date               213
min_view_date                   213
min_cart_date                   213
min_purchase_date               211
cust_retailer_age               214
days_since_last_activity        214
first_view_age                  214
days_since_last_view            214
first_cart_age                  214
days_since_last_cart            214
first_purchase_age              212
days_since_last_purchase        212
dtype: int64

In [18]:
# Grabbing min and max category_spent
max_category_spent = cust_profile.groupby(['user_id','category']).max_category_spent.agg(max_category_spent=np.max).reset_index()
min_category_spent = cust_profile.groupby(['user_id','category']).min_category_spent.agg(min_category_spent=np.min).reset_index()

customer_category_spent = pd.merge(min_category_spent, max_category_spent, on=["user_id","category"], how='inner')
customer_category_spent.head()

,user_id,category,min_category_spent,max_category_spent
0,101875240,2232732085393555647_furniture.living_room.sofa,0.00,0.00
1,101875240,2232732093077520756_construction.tools.light,184.52,184.52
2,107620212,2053013552570040549_electronics.video.projector,0.00,0.00
3,107620212,2053013553165631753_kids.toys,0.00,0.00
4,107620212,2053013553325015316_appliances.kitchen.toster,0.00,0.00


In [19]:
customer_category_spent.nunique()

user_id               1817173
category                  973
min_category_spent      53683
max_category_spent      62020
dtype: int64

In [20]:
# Merging spent and date dfs
customer_agg = pd.merge(customer_category_spent, customer_dates[['user_id','category','cust_retailer_age',
       'days_since_last_activity', 'first_view_age', 'days_since_last_view',
       'first_cart_age', 'days_since_last_cart', 'first_purchase_age',
       'days_since_last_purchase']], on=["user_id","category"], how='inner')
customer_agg.nunique()

user_id                     1817173
category                        959
min_category_spent            51735
max_category_spent            59649
cust_retailer_age               214
days_since_last_activity        214
first_view_age                  214
days_since_last_view            214
first_cart_age                  214
days_since_last_cart            214
first_purchase_age              212
days_since_last_purchase        212
dtype: int64

In [21]:
# Aggregating numeric columns from cust_profile
customer_views = pd.DataFrame(cust_profile.groupby(['user_id','category'])['view_count'].agg('sum')).reset_index()
customer_views.columns = ['user_id','category','total_view']# Renaming columns
customer_carts = pd.DataFrame(cust_profile.groupby(['user_id','category'])['cart_count'].agg('sum')).reset_index()
customer_carts.columns = ['user_id','category','total_cart_add']# Renaming columns
customer_purchases = pd.DataFrame(cust_profile.groupby(['user_id','category'])['purchase_count'].agg('sum')).reset_index()
customer_purchases.columns = ['user_id','category','total_purchases']# Renaming columns
customer_sessions = pd.DataFrame(cust_profile.groupby(['user_id','category'])['session_count'].agg('sum')).reset_index()
customer_sessions.columns = ['user_id','category','total_sessions']# Renaming columns
customer_spent = pd.DataFrame(cust_profile.groupby(['user_id','category'])['spent'].agg('sum')).reset_index()
customer_spent.columns = ['user_id','category','total_spent']# Renaming columns

# Merging event counts dfs together
data_frames = [customer_views, customer_carts, customer_purchases, customer_sessions, customer_spent]
customer_profile = reduce(lambda  left,right: pd.merge(left,right,on=['user_id','category'],how='outer'), data_frames)
customer_profile.nunique()

user_id            1817173
category               973
total_view            1249
total_cart_add         381
total_purchases        292
total_sessions         266
total_spent         362290
dtype: int64

In [22]:
customer_carts.head()

,user_id,category,total_cart_add
0,101875240,2232732085393555647_furniture.living_room.sofa,0.0
1,101875240,2232732093077520756_construction.tools.light,3.0
2,107620212,2053013552570040549_electronics.video.projector,0.0
3,107620212,2053013553165631753_kids.toys,0.0
4,107620212,2053013553325015316_appliances.kitchen.toster,0.0


In [23]:
customer_profile = pd.merge(customer_profile, customer_agg, on=["user_id","category"], how='inner')
customer_profile.head()

,user_id,category,total_view,total_cart_add,total_purchases,total_sessions,total_spent,min_category_spent,max_category_spent,cust_retailer_age,days_since_last_activity,first_view_age,days_since_last_view,first_cart_age,days_since_last_cart,first_purchase_age,days_since_last_purchase
0,101875240,2232732093077520756_construction.tools.light,13.0,3.0,1.0,3.0,184.52,184.52,184.52,105.0,105.0,105.0,105.0,105.0,105.0,105.0,105.0
1,107620212,2053013553325015316_appliances.kitchen.toster,1.0,0.0,0.0,1.0,0.00,0.00,0.00,9999.0,21.0,9999.0,21.0,9999.0,9999.0,9999.0,9999.0
2,107620212,2053013561554240247_electronics.clocks,3.0,0.0,0.0,2.0,0.00,0.00,0.00,136.0,9999.0,136.0,9999.0,9999.0,9999.0,9999.0,9999.0
3,107620212,2232732110022509290_sport.ski,29.0,3.0,1.0,17.0,244.28,0.00,244.28,9999.0,9999.0,9999.0,9999.0,91.0,91.0,91.0,91.0
4,128968633,2053013552351936731_appliances.environment.air...,6.0,0.0,0.0,1.0,0.00,0.00,0.00,174.0,9999.0,174.0,9999.0,9999.0,9999.0,9999.0,9999.0


In [24]:
# Ranking categories for each user based on their views,cart adds, total purchases and total spent
customer_profile['view_rank'] = customer_profile.sort_values(['total_view'],ascending=False).groupby('user_id').cumcount()+1
customer_profile['view_rank'] = customer_profile['view_rank'].astype('int64')

customer_profile['cart_rank'] = customer_profile.sort_values(['total_cart_add'],ascending=False).groupby('user_id').cumcount()+1
customer_profile['cart_rank'] = customer_profile['cart_rank'].astype('int64')

customer_profile['purchases_rank'] = customer_profile.sort_values(['total_purchases'],ascending=False).groupby('user_id').cumcount()+1
customer_profile['purchases_rank'] = customer_profile['purchases_rank'].astype('int64')

customer_profile['spent_rank'] = customer_profile.sort_values(['total_spent'],ascending=False).groupby('user_id').cumcount()+1
customer_profile['spent_rank'] = customer_profile['spent_rank'].astype('int64')


customer_profile.head(10)

,user_id,category,total_view,total_cart_add,total_purchases,total_sessions,total_spent,min_category_spent,max_category_spent,cust_retailer_age,...,first_view_age,days_since_last_view,first_cart_age,days_since_last_cart,first_purchase_age,days_since_last_purchase,view_rank,cart_rank,purchases_rank,spent_rank
0,101875240,2232732093077520756_construction.tools.light,13.0,3.0,1.0,3.0,184.52,184.52,184.52,105.0,...,105.0,105.0,105.0,105.0,105.0,105.0,1,1,1,1
1,107620212,2053013553325015316_appliances.kitchen.toster,1.0,0.0,0.0,1.0,0.00,0.00,0.00,9999.0,...,9999.0,21.0,9999.0,9999.0,9999.0,9999.0,3,3,3,3
2,107620212,2053013561554240247_electronics.clocks,3.0,0.0,0.0,2.0,0.00,0.00,0.00,136.0,...,136.0,9999.0,9999.0,9999.0,9999.0,9999.0,2,2,2,2
3,107620212,2232732110022509290_sport.ski,29.0,3.0,1.0,17.0,244.28,0.00,244.28,9999.0,...,9999.0,9999.0,91.0,91.0,91.0,91.0,1,1,1,1
4,128968633,2053013552351936731_appliances.environment.air...,6.0,0.0,0.0,1.0,0.00,0.00,0.00,174.0,...,174.0,9999.0,9999.0,9999.0,9999.0,9999.0,5,5,5,5
5,128968633,2053013555631882655_electronics.smartphone,16.0,1.0,0.0,4.0,0.00,0.00,0.00,9999.0,...,9999.0,9999.0,160.0,9999.0,9999.0,9999.0,4,4,4,4
6,128968633,2232732093077520756_construction.tools.light,76.0,1.0,0.0,5.0,0.00,0.00,0.00,9999.0,...,9999.0,0.0,9999.0,0.0,9999.0,9999.0,1,3,3,3
7,128968633,2232732102103663163_furniture.bedroom.blanket,17.0,2.0,1.0,1.0,56.60,56.60,56.60,9999.0,...,9999.0,9999.0,9999.0,9999.0,9999.0,121.0,3,2,2,2
8,128968633,2232732108613223108_sport.trainer,20.0,2.0,2.0,3.0,302.19,0.00,157.02,9999.0,...,9999.0,9999.0,9999.0,9999.0,121.0,9999.0,2,1,1,1
9,136662675,2053013553056579841_computers.peripherals.printer,5.0,0.0,0.0,5.0,0.00,0.00,0.00,9999.0,...,9999.0,65.0,9999.0,9999.0,9999.0,9999.0,1,2,2,2


In [25]:
customer_profile.nunique()

user_id                     1817173
category                        959
total_view                     1198
total_cart_add                  369
total_purchases                 280
total_sessions                  240
total_spent                  335866
min_category_spent            51735
max_category_spent            59649
cust_retailer_age               214
days_since_last_activity        214
first_view_age                  214
days_since_last_view            214
first_cart_age                  214
days_since_last_cart            214
first_purchase_age              212
days_since_last_purchase        212
view_rank                         8
cart_rank                         8
purchases_rank                    8
spent_rank                        8
dtype: int64

In [26]:
customer_profile.shape

(4654868, 21)

In [27]:
# Saving results in S3
customer_profile.to_csv('s3://myaws-capstone-bucket/data/category_affinity_profile.csv',index=False)

#### Creating full Brand affinity profile

In [28]:
# Reading customer overall profiles from S3
df_oct = pd.read_csv('s3://myaws-capstone-bucket/data/2019-Oct/customer_brand_profile.csv')
df_nov = pd.read_csv('s3://myaws-capstone-bucket/data/2019-Nov/customer_brand_profile.csv')
df_dec = pd.read_csv('s3://myaws-capstone-bucket/data/2019-Dec/customer_brand_profile.csv')
df_jan = pd.read_csv('s3://myaws-capstone-bucket/data/2020-Jan/customer_brand_profile.csv')
df_feb = pd.read_csv('s3://myaws-capstone-bucket/data/2020-Feb/customer_brand_profile.csv')
df_mar = pd.read_csv('s3://myaws-capstone-bucket/data/2020-Mar/customer_brand_profile.csv')
df_apr = pd.read_csv('s3://myaws-capstone-bucket/data/2020-Apr/customer_brand_profile.csv')

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (13,14) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [29]:
# Concatanating dfs together to create one full customer profile for the 7 months
data_frames = [df_oct, df_nov, df_dec, df_jan, df_feb, df_mar, df_apr]
cust_profile = pd.concat(data_frames)
cust_profile.nunique()

user_id               13699114
brand                     5675
view_count                1206
cart_count                 281
purchase_count             201
session_count              457
spent                   315043
min_brand_spent          62153
max_brand_spent          63654
median_brand_spent      156594
avg_brand_spent         361080
min_activity_date          213
max_activity_date          213
min_view_date              213
max_view_date              213
min_cart_date              213
max_cart_date              213
min_purchase_date          211
max_purchase_date          211
month                        7
dtype: int64

In [30]:
# Merging with customers who have made purchases as these will be our focus customers for the project
cust_profile = pd.merge(cust_profile, main_custs[['user_id']], on=["user_id"], how='inner')
cust_profile.nunique()

user_id               1817173
brand                    5566
view_count                761
cart_count                265
purchase_count            201
session_count             196
spent                  315043
min_brand_spent         62153
max_brand_spent         63654
median_brand_spent     156594
avg_brand_spent        361080
min_activity_date         213
max_activity_date         213
min_view_date             213
max_view_date             213
min_cart_date             213
max_cart_date             213
min_purchase_date         211
max_purchase_date         211
month                       7
dtype: int64

In [31]:
# Rank column based on month customer shopped
cust_profile.loc[(cust_profile['month'] == '2019-Oct'),'month_rank'] = 7
cust_profile.loc[(cust_profile['month'] == '2019-Nov'),'month_rank'] = 6
cust_profile.loc[(cust_profile['month'] == '2019-Dec'),'month_rank'] = 5
cust_profile.loc[(cust_profile['month'] == '2020-Jan'),'month_rank'] = 4
cust_profile.loc[(cust_profile['month'] == '2020-Feb'),'month_rank'] = 3
cust_profile.loc[(cust_profile['month'] == '2020-Mar'),'month_rank'] = 2
cust_profile.loc[(cust_profile['month'] == '2020-Apr'),'month_rank'] = 1

In [32]:
# Saving results with month data added
cust_profile.to_csv('s3://myaws-capstone-bucket/data/brand_profile_full_month_agg.csv',index=False)

In [33]:
# Transforming date columns to datetime type
cust_profile["max_activity_date"] = cust_profile["max_activity_date"].astype('datetime64[ns]')
cust_profile["min_activity_date"] = cust_profile["min_activity_date"].astype('datetime64[ns]')

cust_profile["min_view_date"] = cust_profile["min_view_date"].astype('datetime64[ns]')
cust_profile["max_view_date"] = cust_profile["max_view_date"].astype('datetime64[ns]')

cust_profile["min_cart_date"] = cust_profile["min_cart_date"].astype('datetime64[ns]')
cust_profile["max_cart_date"] = cust_profile["max_cart_date"].astype('datetime64[ns]')

cust_profile["max_purchase_date"] = cust_profile["max_purchase_date"].astype('datetime64[ns]')
cust_profile["min_purchase_date"] = cust_profile["min_purchase_date"].astype('datetime64[ns]')

In [34]:
# Grabbing max dates based on month rank
max_activity_dates = cust_profile.sort_values('max_activity_date',ascending = False).groupby('user_id').head(1)
max_activity_dates = max_activity_dates[['user_id','brand','max_activity_date']]

max_view_dates = cust_profile.sort_values('max_view_date',ascending = False).groupby('user_id').head(1)
max_view_dates = max_view_dates[['user_id','brand','max_view_date']]

max_cart_dates = cust_profile.sort_values('max_cart_date',ascending = False).groupby('user_id').head(1)
max_cart_dates = max_cart_dates[['user_id','brand','max_cart_date']]

max_purchase_dates = cust_profile.sort_values('max_purchase_date',ascending = False).groupby('user_id').head(1)
max_purchase_dates = max_purchase_dates[['user_id','brand','max_purchase_date']]

# merging_max_dates
data_frames = [max_activity_dates,max_view_dates, max_cart_dates, max_purchase_dates]
max_dates = reduce(lambda  left,right: pd.merge(left,right,on=['user_id','brand'],how='outer'), data_frames)

In [35]:
# Grabbing min dates based on month rank
min_activity_dates = cust_profile.sort_values('min_activity_date',ascending = True).groupby('user_id').head(1)
min_activity_dates = min_activity_dates[['user_id','brand','min_activity_date']]

min_view_dates = cust_profile.sort_values('min_view_date',ascending = True).groupby('user_id').head(1)
min_view_dates = min_view_dates[['user_id','brand','min_view_date']]

min_cart_dates = cust_profile.sort_values('min_cart_date',ascending = True).groupby('user_id').head(1)
min_cart_dates = min_cart_dates[['user_id','brand','min_cart_date']]

min_purchase_dates = cust_profile.sort_values('min_purchase_date',ascending = True).groupby('user_id').head(1)
min_purchase_dates = min_purchase_dates[['user_id','brand','min_purchase_date']]

# merging_min_dates
data_frames = [min_activity_dates,min_view_dates, min_cart_dates, min_purchase_dates]
min_dates = reduce(lambda  left,right: pd.merge(left,right,on=['user_id','brand'],how='outer'), data_frames)

In [36]:
# Obtaining max possible date from data 
column = df_apr["max_activity_date"].astype('datetime64[ns]')
max_date = column.max() 
# Transforming date columns to datetime format
max_dates[['max_activity_date','max_view_date','max_cart_date', 'max_purchase_date']] = max_dates[['max_activity_date','max_view_date','max_cart_date', 'max_purchase_date']].astype('datetime64[ns]')
min_dates[['min_activity_date','min_view_date','min_cart_date', 'min_purchase_date']] = min_dates[['min_activity_date','min_view_date','min_cart_date', 'min_purchase_date']].astype('datetime64[ns]')

customer_dates = pd.merge(max_dates, min_dates, on=["user_id","brand"], how='outer')

In [37]:
# Calculating days since each event and customer activity
customer_dates['cust_retailer_age'] = (max_date - customer_dates['min_activity_date']).dt.days
customer_dates['days_since_last_activity'] = (max_date - customer_dates['max_activity_date']).dt.days

customer_dates['first_view_age'] = (max_date - customer_dates['min_view_date']).dt.days
customer_dates['days_since_last_view'] = (max_date - customer_dates['max_view_date']).dt.days

customer_dates['first_cart_age'] = (max_date - customer_dates['min_cart_date']).dt.days
customer_dates['days_since_last_cart'] = (max_date - customer_dates['max_cart_date']).dt.days


customer_dates['first_purchase_age'] = (max_date - customer_dates['min_purchase_date']).dt.days
customer_dates['days_since_last_purchase'] = (max_date - customer_dates['max_purchase_date']).dt.days

customer_dates[['cust_retailer_age',
       'days_since_last_activity', 'first_view_age', 'days_since_last_view',
       'first_cart_age', 'days_since_last_cart', 'first_purchase_age',
       'days_since_last_purchase']] = customer_dates[['cust_retailer_age',
       'days_since_last_activity', 'first_view_age', 'days_since_last_view',
       'first_cart_age', 'days_since_last_cart', 'first_purchase_age',
       'days_since_last_purchase']].fillna(value=9999)# Replacing nan with 9999 to symbolize they have not done said action

customer_dates.head()

,user_id,brand,max_activity_date,max_view_date,max_cart_date,max_purchase_date,min_activity_date,min_view_date,min_cart_date,min_purchase_date,cust_retailer_age,days_since_last_activity,first_view_age,days_since_last_view,first_cart_age,days_since_last_cart,first_purchase_age,days_since_last_purchase
0,649742534,samsung,2020-04-30,NaT,NaT,2020-04-30,2020-04-30,NaT,NaT,2020-04-30,0.0,0.0,9999.0,9999.0,9999.0,9999.0,0.0,0.0
1,516016539,garmin,2020-04-30,NaT,NaT,NaT,2019-10-02,NaT,NaT,NaT,211.0,0.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0
2,641706454,tcl,2020-04-30,2020-04-30,2020-04-30,2020-04-30,NaT,NaT,NaT,2020-04-30,9999.0,0.0,9999.0,0.0,9999.0,0.0,0.0,0.0
3,528443799,teknum,2020-04-30,NaT,NaT,NaT,NaT,NaT,NaT,NaT,9999.0,0.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0
4,516016596,ariston,2020-04-30,NaT,2020-04-30,2020-04-30,NaT,NaT,NaT,NaT,9999.0,0.0,9999.0,9999.0,9999.0,0.0,9999.0,0.0


In [38]:
# Grabbing min and max brand_spent
max_brand_spent = cust_profile.groupby(['user_id','brand']).max_brand_spent.agg(max_brand_spent=np.max).reset_index()
min_brand_spent = cust_profile.groupby(['user_id','brand']).min_brand_spent.agg(min_brand_spent=np.min).reset_index()

customer_brand_spent = pd.merge(min_brand_spent, max_brand_spent, on=["user_id","brand"], how='inner')
customer_brand_spent.head()

,user_id,brand,min_brand_spent,max_brand_spent
0,101875240,gehwol,0.00,0.00
1,101875240,samsung,0.00,0.00
2,101875240,xiaomi,184.52,184.52
3,107620212,arg,0.00,0.00
4,107620212,blackbox,0.00,0.00


In [39]:
# Merging spent and date dfs
customer_agg = pd.merge(customer_brand_spent, customer_dates[['user_id','brand','cust_retailer_age',
       'days_since_last_activity', 'first_view_age', 'days_since_last_view',
       'first_cart_age', 'days_since_last_cart', 'first_purchase_age',
       'days_since_last_purchase']], on=["user_id","brand"], how='inner')
customer_agg.nunique()

user_id                     1817173
brand                          4949
min_brand_spent               50361
max_brand_spent               59525
cust_retailer_age               214
days_since_last_activity        214
first_view_age                  214
days_since_last_view            214
first_cart_age                  214
days_since_last_cart            214
first_purchase_age              212
days_since_last_purchase        212
dtype: int64

In [40]:
# Aggregating numeric columns from cust_profile
customer_views = pd.DataFrame(cust_profile.groupby(['user_id','brand'])['view_count'].agg('sum')).reset_index()
customer_views.columns = ['user_id','brand','total_view']# Renaming columns
customer_carts = pd.DataFrame(cust_profile.groupby(['user_id','brand'])['cart_count'].agg('sum')).reset_index()
customer_carts.columns = ['user_id','brand','total_cart_add']# Renaming columns
customer_purchases = pd.DataFrame(cust_profile.groupby(['user_id','brand'])['purchase_count'].agg('sum')).reset_index()
customer_purchases.columns = ['user_id','brand','total_purchases']# Renaming columns
customer_sessions = pd.DataFrame(cust_profile.groupby(['user_id','brand'])['session_count'].agg('sum')).reset_index()
customer_sessions.columns = ['user_id','brand','total_sessions']# Renaming columns
customer_spent = pd.DataFrame(cust_profile.groupby(['user_id','brand'])['spent'].agg('sum')).reset_index()
customer_spent.columns = ['user_id','brand','total_spent']# Renaming columns

# Merging event counts dfs together
data_frames = [customer_views, customer_carts, customer_purchases, customer_sessions, customer_spent]
customer_profile = reduce(lambda  left,right: pd.merge(left,right,on=['user_id','brand'],how='outer'), data_frames)
customer_profile.nunique()

user_id            1817173
brand                 5566
total_view            1045
total_cart_add         368
total_purchases        290
total_sessions         205
total_spent         357436
dtype: int64

In [41]:
customer_profile = pd.merge(customer_profile, customer_agg, on=["user_id","brand"], how='inner')
customer_profile.head()

,user_id,brand,total_view,total_cart_add,total_purchases,total_sessions,total_spent,min_brand_spent,max_brand_spent,cust_retailer_age,days_since_last_activity,first_view_age,days_since_last_view,first_cart_age,days_since_last_cart,first_purchase_age,days_since_last_purchase
0,101875240,gehwol,2.0,0.0,0.0,2.0,0.00,0.00,0.00,9999.0,9999.0,9999.0,105.0,9999.0,9999.0,9999.0,9999.0
1,101875240,xiaomi,9.0,3.0,1.0,13.0,184.52,184.52,184.52,105.0,105.0,105.0,9999.0,105.0,105.0,105.0,105.0
2,107620212,blackbox,2.0,0.0,0.0,1.0,0.00,0.00,0.00,9999.0,9999.0,136.0,9999.0,9999.0,9999.0,9999.0,9999.0
3,107620212,elan,17.0,3.0,1.0,9.0,244.28,0.00,244.28,9999.0,9999.0,9999.0,9999.0,91.0,91.0,91.0,91.0
4,107620212,phoenix,1.0,0.0,0.0,1.0,0.00,0.00,0.00,9999.0,21.0,9999.0,21.0,9999.0,9999.0,9999.0,9999.0


In [42]:
# Ranking categories for each user based on their views,cart adds, total purchases and total spent
customer_profile['view_rank'] = customer_profile.sort_values(['total_view'],ascending=False).groupby('user_id').cumcount()+1
customer_profile['view_rank'] = customer_profile['view_rank'].astype('int64')

customer_profile['cart_rank'] = customer_profile.sort_values(['total_cart_add'],ascending=False).groupby('user_id').cumcount()+1
customer_profile['cart_rank'] = customer_profile['cart_rank'].astype('int64')

customer_profile['purchases_rank'] = customer_profile.sort_values(['total_purchases'],ascending=False).groupby('user_id').cumcount()+1
customer_profile['purchases_rank'] = customer_profile['purchases_rank'].astype('int64')

customer_profile['spent_rank'] = customer_profile.sort_values(['total_spent'],ascending=False).groupby('user_id').cumcount()+1
customer_profile['spent_rank'] = customer_profile['spent_rank'].astype('int64')


customer_profile.head(10)

,user_id,brand,total_view,total_cart_add,total_purchases,total_sessions,total_spent,min_brand_spent,max_brand_spent,cust_retailer_age,...,first_view_age,days_since_last_view,first_cart_age,days_since_last_cart,first_purchase_age,days_since_last_purchase,view_rank,cart_rank,purchases_rank,spent_rank
0,101875240,gehwol,2.0,0.0,0.0,2.0,0.00,0.00,0.00,9999.0,...,9999.0,105.0,9999.0,9999.0,9999.0,9999.0,2,2,2,2
1,101875240,xiaomi,9.0,3.0,1.0,13.0,184.52,184.52,184.52,105.0,...,105.0,9999.0,105.0,105.0,105.0,105.0,1,1,1,1
2,107620212,blackbox,2.0,0.0,0.0,1.0,0.00,0.00,0.00,9999.0,...,136.0,9999.0,9999.0,9999.0,9999.0,9999.0,2,2,2,2
3,107620212,elan,17.0,3.0,1.0,9.0,244.28,0.00,244.28,9999.0,...,9999.0,9999.0,91.0,91.0,91.0,91.0,1,1,1,1
4,107620212,phoenix,1.0,0.0,0.0,1.0,0.00,0.00,0.00,9999.0,...,9999.0,21.0,9999.0,9999.0,9999.0,9999.0,4,3,3,3
5,107620212,triumf,1.0,0.0,0.0,1.0,0.00,0.00,0.00,136.0,...,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,3,4,4,4
6,128968633,dauscher,32.0,2.0,1.0,3.0,56.60,0.00,56.60,9999.0,...,9999.0,9999.0,9999.0,9999.0,121.0,121.0,3,1,1,1
7,128968633,samsung,50.0,1.0,0.0,8.0,0.00,0.00,0.00,9999.0,...,9999.0,9999.0,160.0,9999.0,9999.0,9999.0,1,2,2,2
8,128968633,tcl,2.0,0.0,0.0,1.0,0.00,0.00,0.00,174.0,...,174.0,9999.0,9999.0,9999.0,9999.0,9999.0,4,4,3,3
9,128968633,xiaomi,40.0,1.0,0.0,8.0,0.00,0.00,0.00,9999.0,...,9999.0,0.0,9999.0,0.0,9999.0,9999.0,2,3,4,4


In [43]:
customer_profile.nunique()

user_id                     1817173
brand                          4949
total_view                      991
total_cart_add                  364
total_purchases                 284
total_sessions                  194
total_spent                  331114
min_brand_spent               50361
max_brand_spent               59525
cust_retailer_age               214
days_since_last_activity        214
first_view_age                  214
days_since_last_view            214
first_cart_age                  214
days_since_last_cart            214
first_purchase_age              212
days_since_last_purchase        212
view_rank                         8
cart_rank                         8
purchases_rank                    8
spent_rank                        8
dtype: int64

In [44]:
customer_profile.shape

(5094711, 21)

In [45]:
# Saving results in S3
customer_profile.to_csv('s3://myaws-capstone-bucket/data/brand_affinity_profile.csv',index=False)

#### Creating full product affinity profile

In [46]:
# Reading customer overall profiles from S3
df_oct = pd.read_csv('s3://myaws-capstone-bucket/data/2019-Oct/customer_product_profile.csv')
df_oct = pd.merge(df_oct, main_custs[['user_id']], on=["user_id"], how='inner')

df_nov = pd.read_csv('s3://myaws-capstone-bucket/data/2019-Nov/customer_product_profile.csv')
df_nov = pd.merge(df_nov, main_custs[['user_id']], on=["user_id"], how='inner')

df_dec = pd.read_csv('s3://myaws-capstone-bucket/data/2019-Dec/customer_product_profile.csv')
df_dec = pd.merge(df_dec, main_custs[['user_id']], on=["user_id"], how='inner')

df_jan = pd.read_csv('s3://myaws-capstone-bucket/data/2020-Jan/customer_product_profile.csv')
df_jan = pd.merge(df_jan, main_custs[['user_id']], on=["user_id"], how='inner')

df_feb = pd.read_csv('s3://myaws-capstone-bucket/data/2020-Feb/customer_product_profile.csv')
df_feb = pd.merge(df_feb, main_custs[['user_id']], on=["user_id"], how='inner')

df_mar = pd.read_csv('s3://myaws-capstone-bucket/data/2020-Mar/customer_product_profile.csv')
df_mar = pd.merge(df_mar, main_custs[['user_id']], on=["user_id"], how='inner')

df_apr = pd.read_csv('s3://myaws-capstone-bucket/data/2020-Apr/customer_product_profile.csv')
df_apr = pd.merge(df_apr, main_custs[['user_id']], on=["user_id"], how='inner')

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (13,14) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [47]:
# Concatanating dfs together to create one full customer profile for the 7 months
data_frames = [df_oct, df_nov, df_dec, df_jan, df_feb, df_mar, df_apr]
cust_profile = pd.concat(data_frames)
cust_profile.nunique()

user_id                 1817173
product                  267633
view_count                  278
cart_count                  205
purchase_count              139
session_count               144
spent                    219604
min_product_spent         66002
max_product_spent         65721
median_product_spent     114475
avg_product_spent        216452
min_activity_date           213
max_activity_date           213
min_view_date               213
max_view_date               213
min_cart_date               213
max_cart_date               213
min_purchase_date           211
max_purchase_date           211
month                         7
dtype: int64

In [48]:
# Rank column based on month customer shopped
cust_profile.loc[(cust_profile['month'] == '2019-Oct'),'month_rank'] = 7
cust_profile.loc[(cust_profile['month'] == '2019-Nov'),'month_rank'] = 6
cust_profile.loc[(cust_profile['month'] == '2019-Dec'),'month_rank'] = 5
cust_profile.loc[(cust_profile['month'] == '2020-Jan'),'month_rank'] = 4
cust_profile.loc[(cust_profile['month'] == '2020-Feb'),'month_rank'] = 3
cust_profile.loc[(cust_profile['month'] == '2020-Mar'),'month_rank'] = 2
cust_profile.loc[(cust_profile['month'] == '2020-Apr'),'month_rank'] = 1

In [49]:
# Saving results with month data added
cust_profile.to_csv('s3://myaws-capstone-bucket/data/product_profile_full_month_agg.csv',index=False)

In [50]:
# Transforming date columns to datetime type
cust_profile["max_activity_date"] = cust_profile["max_activity_date"].astype('datetime64[ns]')
cust_profile["min_activity_date"] = cust_profile["min_activity_date"].astype('datetime64[ns]')

cust_profile["min_view_date"] = cust_profile["min_view_date"].astype('datetime64[ns]')
cust_profile["max_view_date"] = cust_profile["max_view_date"].astype('datetime64[ns]')

cust_profile["min_cart_date"] = cust_profile["min_cart_date"].astype('datetime64[ns]')
cust_profile["max_cart_date"] = cust_profile["max_cart_date"].astype('datetime64[ns]')

cust_profile["max_purchase_date"] = cust_profile["max_purchase_date"].astype('datetime64[ns]')
cust_profile["min_purchase_date"] = cust_profile["min_purchase_date"].astype('datetime64[ns]')

In [51]:
# Grabbing max dates based on month rank
max_activity_dates = cust_profile.sort_values('max_activity_date',ascending = False).groupby('user_id').head(1)
max_activity_dates = max_activity_dates[['user_id','product','max_activity_date']]

max_view_dates = cust_profile.sort_values('max_view_date',ascending = False).groupby('user_id').head(1)
max_view_dates = max_view_dates[['user_id','product','max_view_date']]

max_cart_dates = cust_profile.sort_values('max_cart_date',ascending = False).groupby('user_id').head(1)
max_cart_dates = max_cart_dates[['user_id','product','max_cart_date']]

max_purchase_dates = cust_profile.sort_values('max_purchase_date',ascending = False).groupby('user_id').head(1)
max_purchase_dates = max_purchase_dates[['user_id','product','max_purchase_date']]

# merging_max_dates
data_frames = [max_activity_dates,max_view_dates, max_cart_dates, max_purchase_dates]
max_dates = reduce(lambda  left,right: pd.merge(left,right,on=['user_id','product'],how='outer'), data_frames)

In [52]:
# Grabbing min dates based on month rank
min_activity_dates = cust_profile.sort_values('min_activity_date',ascending = True).groupby('user_id').head(1)
min_activity_dates = min_activity_dates[['user_id','product','min_activity_date']]

min_view_dates = cust_profile.sort_values('min_view_date',ascending = True).groupby('user_id').head(1)
min_view_dates = min_view_dates[['user_id','product','min_view_date']]

min_cart_dates = cust_profile.sort_values('min_cart_date',ascending = True).groupby('user_id').head(1)
min_cart_dates = min_cart_dates[['user_id','product','min_cart_date']]

min_purchase_dates = cust_profile.sort_values('min_purchase_date',ascending = True).groupby('user_id').head(1)
min_purchase_dates = min_purchase_dates[['user_id','product','min_purchase_date']]

# merging_min_dates
data_frames = [min_activity_dates,min_view_dates, min_cart_dates, min_purchase_dates]
min_dates = reduce(lambda  left,right: pd.merge(left,right,on=['user_id','product'],how='outer'), data_frames)

In [53]:
# Obtaining max possible date from data 
column = df_apr["max_activity_date"].astype('datetime64[ns]')
max_date = column.max() 
# Transforming date columns to datetime format
max_dates[['max_activity_date','max_view_date','max_cart_date', 'max_purchase_date']] = max_dates[['max_activity_date','max_view_date','max_cart_date', 'max_purchase_date']].astype('datetime64[ns]')
min_dates[['min_activity_date','min_view_date','min_cart_date', 'min_purchase_date']] = min_dates[['min_activity_date','min_view_date','min_cart_date', 'min_purchase_date']].astype('datetime64[ns]')

customer_dates = pd.merge(max_dates, min_dates, on=["user_id","product"], how='outer')

In [54]:
# Calculating days since each event and customer activity
customer_dates['cust_retailer_age'] = (max_date - customer_dates['min_activity_date']).dt.days
customer_dates['days_since_last_activity'] = (max_date - customer_dates['max_activity_date']).dt.days

customer_dates['first_view_age'] = (max_date - customer_dates['min_view_date']).dt.days
customer_dates['days_since_last_view'] = (max_date - customer_dates['max_view_date']).dt.days

customer_dates['first_cart_age'] = (max_date - customer_dates['min_cart_date']).dt.days
customer_dates['days_since_last_cart'] = (max_date - customer_dates['max_cart_date']).dt.days


customer_dates['first_purchase_age'] = (max_date - customer_dates['min_purchase_date']).dt.days
customer_dates['days_since_last_purchase'] = (max_date - customer_dates['max_purchase_date']).dt.days

customer_dates[['cust_retailer_age',
       'days_since_last_activity', 'first_view_age', 'days_since_last_view',
       'first_cart_age', 'days_since_last_cart', 'first_purchase_age',
       'days_since_last_purchase']] = customer_dates[['cust_retailer_age',
       'days_since_last_activity', 'first_view_age', 'days_since_last_view',
       'first_cart_age', 'days_since_last_cart', 'first_purchase_age',
       'days_since_last_purchase']].fillna(value=9999)# Replacing nan with 9999 to symbolize they have not done said action

customer_dates.head()

,user_id,product,max_activity_date,max_view_date,max_cart_date,max_purchase_date,min_activity_date,min_view_date,min_cart_date,min_purchase_date,cust_retailer_age,days_since_last_activity,first_view_age,days_since_last_view,first_cart_age,days_since_last_cart,first_purchase_age,days_since_last_purchase
0,649742534,1005100_construction.tools.light,2020-04-30,NaT,NaT,2020-04-30,2020-04-30,NaT,NaT,2020-04-30,0.0,0.0,9999.0,9999.0,9999.0,9999.0,0.0,0.0
1,519888513,1003712_construction.tools.light,2020-04-30,NaT,NaT,NaT,NaT,NaT,NaT,NaT,9999.0,0.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0
2,640929176,100058915_apparel.trousers,2020-04-30,2020-04-30,2020-04-30,2020-04-30,NaT,NaT,NaT,NaT,9999.0,0.0,9999.0,0.0,9999.0,0.0,9999.0,0.0
3,519891401,100086238_construction.tools.light,2020-04-30,2020-04-30,NaT,NaT,NaT,NaT,NaT,NaT,9999.0,0.0,9999.0,0.0,9999.0,9999.0,9999.0,9999.0
4,594466934,47000067_computers.peripherals.mouse,2020-04-30,NaT,NaT,NaT,NaT,NaT,NaT,NaT,9999.0,0.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0


In [55]:
# Grabbing min and max product_spent
max_product_spent = cust_profile.groupby(['user_id','product']).max_product_spent.agg(max_product_spent=np.max).reset_index()
min_product_spent = cust_profile.groupby(['user_id','product']).min_product_spent.agg(min_product_spent=np.min).reset_index()

customer_product_spent = pd.merge(min_product_spent, max_product_spent, on=["user_id","product"], how='inner')
customer_product_spent.head()

,user_id,product,min_product_spent,max_product_spent
0,101875240,100019252_construction.tools.light,184.52,184.52
1,101875240,100020739_construction.tools.light,0.00,0.00
2,101875240,100041313_construction.tools.light,0.00,0.00
3,101875240,1004856_construction.tools.light,0.00,0.00
4,101875240,1005100_construction.tools.light,0.00,0.00


In [56]:
# Merging spent and date dfs
customer_agg = pd.merge(customer_product_spent, customer_dates[['user_id','product','cust_retailer_age',
       'days_since_last_activity', 'first_view_age', 'days_since_last_view',
       'first_cart_age', 'days_since_last_cart', 'first_purchase_age',
       'days_since_last_purchase']], on=["user_id","product"], how='inner')
customer_agg.nunique()

user_id                     1817173
product                      164566
min_product_spent             57397
max_product_spent             59576
cust_retailer_age               214
days_since_last_activity        214
first_view_age                  214
days_since_last_view            214
first_cart_age                  214
days_since_last_cart            214
first_purchase_age              212
days_since_last_purchase        212
dtype: int64

In [57]:
# Aggregating numeric columns from cust_profile
customer_views = pd.DataFrame(cust_profile.groupby(['user_id','product'])['view_count'].agg('sum')).reset_index()
customer_views.columns = ['user_id','product','total_view']# Renaming columns
customer_carts = pd.DataFrame(cust_profile.groupby(['user_id','product'])['cart_count'].agg('sum')).reset_index()
customer_carts.columns = ['user_id','product','total_cart_add']# Renaming columns
customer_purchases = pd.DataFrame(cust_profile.groupby(['user_id','product'])['purchase_count'].agg('sum')).reset_index()
customer_purchases.columns = ['user_id','product','total_purchases']# Renaming columns
customer_sessions = pd.DataFrame(cust_profile.groupby(['user_id','product'])['session_count'].agg('sum')).reset_index()
customer_sessions.columns = ['user_id','product','total_sessions']# Renaming columns
customer_spent = pd.DataFrame(cust_profile.groupby(['user_id','product'])['spent'].agg('sum')).reset_index()
customer_spent.columns = ['user_id','product','total_spent']# Renaming columns

# Merging event counts dfs together
data_frames = [customer_views, customer_carts, customer_purchases, customer_sessions, customer_spent]
customer_profile = reduce(lambda  left,right: pd.merge(left,right,on=['user_id','product'],how='outer'), data_frames)
customer_profile.nunique()

user_id            1817173
product             267633
total_view             426
total_cart_add         239
total_purchases        162
total_sessions         147
total_spent         251657
dtype: int64

In [58]:
customer_profile = pd.merge(customer_profile, customer_agg, on=["user_id","product"], how='inner')
customer_profile.head()

,user_id,product,total_view,total_cart_add,total_purchases,total_sessions,total_spent,min_product_spent,max_product_spent,cust_retailer_age,days_since_last_activity,first_view_age,days_since_last_view,first_cart_age,days_since_last_cart,first_purchase_age,days_since_last_purchase
0,101875240,100019252_construction.tools.light,2.0,2.0,1.0,5.0,184.52,184.52,184.52,9999.0,9999.0,9999.0,105.0,9999.0,9999.0,105.0,105.0
1,101875240,1005100_construction.tools.light,1.0,0.0,0.0,1.0,0.00,0.00,0.00,9999.0,105.0,105.0,9999.0,9999.0,9999.0,9999.0,9999.0
2,101875240,1005161_construction.tools.light,4.0,1.0,0.0,5.0,0.00,0.00,0.00,9999.0,9999.0,9999.0,9999.0,105.0,105.0,9999.0,9999.0
3,101875240,17601362_furniture.living_room.sofa,2.0,0.0,0.0,2.0,0.00,0.00,0.00,105.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0
4,107620212,100013581_electronics.clocks,1.0,0.0,0.0,1.0,0.00,0.00,0.00,9999.0,9999.0,136.0,9999.0,9999.0,9999.0,9999.0,9999.0


In [59]:
# Ranking categories for each user based on their views,cart adds, total purchases and total spent
customer_profile['view_rank'] = customer_profile.sort_values(['total_view'],ascending=False).groupby('user_id').cumcount()+1
customer_profile['view_rank'] = customer_profile['view_rank'].astype('int64')

customer_profile['cart_rank'] = customer_profile.sort_values(['total_cart_add'],ascending=False).groupby('user_id').cumcount()+1
customer_profile['cart_rank'] = customer_profile['cart_rank'].astype('int64')

customer_profile['purchases_rank'] = customer_profile.sort_values(['total_purchases'],ascending=False).groupby('user_id').cumcount()+1
customer_profile['purchases_rank'] = customer_profile['purchases_rank'].astype('int64')

customer_profile['spent_rank'] = customer_profile.sort_values(['total_spent'],ascending=False).groupby('user_id').cumcount()+1
customer_profile['spent_rank'] = customer_profile['spent_rank'].astype('int64')


customer_profile.head(10)

,user_id,product,total_view,total_cart_add,total_purchases,total_sessions,total_spent,min_product_spent,max_product_spent,cust_retailer_age,...,first_view_age,days_since_last_view,first_cart_age,days_since_last_cart,first_purchase_age,days_since_last_purchase,view_rank,cart_rank,purchases_rank,spent_rank
0,101875240,100019252_construction.tools.light,2.0,2.0,1.0,5.0,184.52,184.52,184.52,9999.0,...,9999.0,105.0,9999.0,9999.0,105.0,105.0,2,1,1,1
1,101875240,1005100_construction.tools.light,1.0,0.0,0.0,1.0,0.00,0.00,0.00,9999.0,...,105.0,9999.0,9999.0,9999.0,9999.0,9999.0,4,4,4,4
2,101875240,1005161_construction.tools.light,4.0,1.0,0.0,5.0,0.00,0.00,0.00,9999.0,...,9999.0,9999.0,105.0,105.0,9999.0,9999.0,1,2,3,2
3,101875240,17601362_furniture.living_room.sofa,2.0,0.0,0.0,2.0,0.00,0.00,0.00,105.0,...,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,3,3,2,3
4,107620212,100013581_electronics.clocks,1.0,0.0,0.0,1.0,0.00,0.00,0.00,9999.0,...,136.0,9999.0,9999.0,9999.0,9999.0,9999.0,4,3,4,3
5,107620212,100015096_electronics.clocks,1.0,0.0,0.0,1.0,0.00,0.00,0.00,136.0,...,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,3,2,3,4
6,107620212,12202300_appliances.kitchen.toster,1.0,0.0,0.0,1.0,0.00,0.00,0.00,9999.0,...,9999.0,21.0,9999.0,9999.0,9999.0,9999.0,2,4,2,2
7,107620212,20500418_sport.ski,6.0,3.0,1.0,1.0,244.28,244.28,244.28,9999.0,...,9999.0,9999.0,91.0,91.0,91.0,91.0,1,1,1,1
8,128968633,100009754_construction.tools.light,3.0,1.0,0.0,3.0,0.00,0.00,0.00,9999.0,...,9999.0,9999.0,9999.0,0.0,9999.0,9999.0,3,4,7,6
9,128968633,1004888_electronics.smartphone,3.0,1.0,0.0,3.0,0.00,0.00,0.00,9999.0,...,9999.0,9999.0,160.0,9999.0,9999.0,9999.0,4,3,6,7


In [60]:
customer_profile.nunique()

user_id                     1817173
product                      164566
total_view                      279
total_cart_add                  210
total_purchases                 127
total_sessions                  135
total_spent                  188881
min_product_spent             57397
max_product_spent             59576
cust_retailer_age               214
days_since_last_activity        214
first_view_age                  214
days_since_last_view            214
first_cart_age                  214
days_since_last_cart            214
first_purchase_age              212
days_since_last_purchase        212
view_rank                         8
cart_rank                         8
purchases_rank                    8
spent_rank                        8
dtype: int64

In [61]:
# Saving results in S3
customer_profile.to_csv('s3://myaws-capstone-bucket/data/product_affinity_profile.csv',index=False)